# Brightway2 seminar
Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Day 1, afternoon  
Session on working with `Activity` and ` Exchange` proxies 

### Learning objectives  
  - Work with `Activity` and ` Exchange` proxies (copy, add, modify, delete)  
  - (OPTIONAL - CHRIS?) Calculate uncertainty based on Pedigree  

**Exercise**: Create a new activity for offset printing in Quebec (CA-QC) that is based on a copy of the activity `'offset printing, per kg printed paper' (kilogram, CH, None)`. It is to have the following modifications:  
  - Its location is to be changed to "CA-QC"  
  - Its source of electricity is to be changed to electricity from the market for low-voltage electricity in CA-QC  
  - Suppose the printing generates no waste. Its paper input needs to be set to 1kg, and its waste paper output needs to be deleted. 

CHRIS: this may be fun, but probably would take too long.
  - The Pedigree matrix scores for the exchanges are to be modified  
  - The uncertainty is to be modified based on these modified Pedigree scores

**Step 1**: Import Brightway as bw, open the seminar project and make a copy of the ecoinvent v3.3 database (this step can be long, let's do it before the break).

In [1]:
import brightway2 as bw

In [2]:
# bw.projects.set_current('bw2_seminar_2017')
bw.projects.set_current("bw2 seminar 2017 😄")

In [3]:
if 'ecoinvent copy' not in bw.databases:
    bw.Database('ecoinvent').copy('ecoinvent copy')
else:
    print('A copy already exists')
eidb = bw.Database('ecoinvent copy')

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:26


Title: Writing activities to SQLite3 database:
  Started: 03/27/2017 23:37:23
  Finished: 03/27/2017 23:37:49
  Total time elapsed: 00:00:26
  CPU %: 97.20
  Memory %: 3.03


**Step 2**: Create a copy of the activity `'offset printing, per kg printed paper' (kilogram, CH, None)` and assign it to a variable.  

In [23]:
# Solution
CH_printing = [act for act in eidb if "print" in act['name']]
# QC_printing = CH_printing.copy()
CH_printing

['production of carton board boxes, gravure printing, at plant' (kilogram, CH, ['paper & cardboard', 'cardboard & corrugated board']),
 'printed wiring board, surface mount, at plant' (square meter, GLO, ['electronics', 'module']),
 'printed wiring board, mixed mounted, unspec., solder mix, at plant' (kilogram, GLO, ['electronics', 'module']),
 'printer, laser jet, colour, at plant' (unit, GLO, ['electronics', 'devices']),
 'paper, newsprint, at regional storage' (kilogram, RER, ['paper & cardboard', 'graphic paper']),
 'dismantling, printer, laser, mechanically, at plant' (kilogram, GLO, ['waste management', 'recycling']),
 'Fe fraction, mechanical treatment, printer, laser, at plant' (kilogram, GLO, ['waste management', 'recycling']),
 'production of carton board boxes, offset printing, at plant' (kilogram, CH, ['paper & cardboard', 'cardboard & corrugated board']),
 'printed wiring board, through-hole, lead-containing surface, at plant' (square meter, GLO, ['electronics', 'module'])

**Step 3**: Just a check: make sure that the key of the activities are not the same

In [5]:
CH_printing.key == QC_printing.key

False

**Step 4**: Change the QC activity's location to "CA-QC"

In [6]:
QC_printing['location']="CA-QC"
QC_printing.save()

**Step 5**: Find the electricity exchange to be replaced in the Quebec printing activity.

In [7]:
electricity_input_exchange = [exc for exc in QC_printing.technosphere() if "electricity" in exc['name']][0]
electricity_input_exchange

Exchange: 0.325 kilowatt hour 'electricity, medium voltage, at grid' (kilowatt hour, CH, ['electricity', 'supply mix']) to 'production of carton board boxes, offset printing, at plant' (kilogram, CA-QC, ['paper & cardboard', 'cardboard & corrugated board'])>

**Step 6**: Find the activity that this exchange needs to be relinked to.

In [12]:
qc_elect=[act for act in eidb 
    if 'electricity' in act['name']
    and 'low voltage' in act['name']
    and act['location'] == 'BR'  # QC not in 2.2
][0]

'electricity, low voltage, production BR, at grid' (kilowatt hour, BR, ['electricity', 'production mix'])

In [ ]:
qc_elect

**Step 7**: Relink the electricity input exchange to the QC electricity:

In [13]:
electricity_input_exchange.input = qc_elect
electricity_input_exchange.save()

**Step 8**: Delete the waste paper output.

# TODO: Update for 2.2

In [14]:
waste_paper_output = [exc for exc in QC_printing.technosphere() if "paper" in exc['name'] and exc['amount'] < 0][0]
waste_paper_output

IndexError: list index out of range

In [12]:
waste_paper_output.delete()

**Step 9**: Change the paper input amount to 1 kg.

In [16]:
paper_input = [exc for exc in QC_printing.technosphere() if "paper, woodfree" in exc['name']][0]
paper_input

Exchange: 0.0543 kilogram 'paper, woodfree, coated, at regional storage' (kilogram, RER, ['paper & cardboard', 'graphic paper']) to 'production of carton board boxes, offset printing, at plant' (kilogram, CA-QC, ['paper & cardboard', 'cardboard & corrugated board'])>

In [18]:
QC_printing

'production of carton board boxes, offset printing, at plant' (kilogram, CA-QC, ['paper & cardboard', 'cardboard & corrugated board'])

In [14]:
paper_input['amount']=1
paper_input.save()

In [15]:
QC_printing.save()

**Final result:** 

In [25]:
for exc in QC_printing.exchanges():
    print(exc.input['name'], exc['amount'], exc['unit'])

chemicals organic, at plant 0.00023 kilogram
lubricating oil, at plant 0.0002 kilogram
rosin size, in paper production, at plant 0.0052 kilogram
solvents, organic, unspecified, at plant 0.0022 kilogram
electricity, low voltage, production BR, at grid 0.325 kilowatt hour
natural gas, burned in industrial furnace >100kW 1.19 megajoule
propane/ butane, at refinery 0.0001 kilogram
light fuel oil, burned in industrial furnace 1MW, non-modulating 0.682 megajoule
printing colour, offset, 47.5% solvent, at plant 0.0183 kilogram
packaging box production unit 1.43e-09 unit
production of carton board boxes, offset printing, at plant 1.0 kilogram
paper, woodfree, coated, at regional storage 0.0543 kilogram
polyethylene, HDPE, granulate, at plant 0.0108 kilogram
transport, lorry >16t, fleet average 0.00369 ton kilometer
transport, van <3.5t 0.00621 ton kilometer
transport, freight, rail 0.0222 ton kilometer
disposal, bilge oil, 90% water, to hazardous waste incineration 0.00016 kilogram
disposal, m

In [ ]:
for exc in CH_printing.exchanges():
    print(exc.input['name'], exc['amount'], exc['unit'])